In [1]:
import pandas as pd
import numpy as np
import sys, json, os

In [2]:
val=pd.read_json("val.json", encoding = 'utf-8')
result=pd.read_json("results.json", encoding ='utf-8')

In [3]:
def check_h(val, result, n=5):
    print(val.head(n))
    print(result.head(n))
    print("val",val.shape)
    print("result",result.shape)
def check_t(val, result, n=5):
    print(val.tail(n))
    print(result.tail(n))
    print("val",val.shape)
    print("result",result.shape)

In [4]:
check_h(val, result)

       id  like_cnt plylst_title  \
0  118598      1675                
1  131447         1       앨리스테이블   
2   51464        62                
3   45144        20                
4   79929        20                

                                               songs tags  \
0  [373313, 151080, 275346, 696876, 165237, 52593...   []   
1                                                 []   []   
2  [529437, 516103, 360067, 705713, 226062, 37089...   []   
3  [589668, 21711, 570151, 320043, 13930, 599327,...   []   
4  [672718, 121924, 102694, 683657, 201558, 38511...   []   

                 updt_date  
0  2019-05-27 14:14:33.000  
1  2014-07-16 15:24:24.000  
2  2008-06-21 23:26:22.000  
3  2017-10-30 18:15:43.000  
4  2017-02-07 11:40:42.000  
       id                                              songs  \
0  118598  [207912, 623047, 422438, 703323, 638488, 32221...   
1  131447  [16382, 278524, 196599, 442352, 704484, 589785...   
2   51464  [456474, 533739, 680911, 596279, 425043

In [5]:
val_map=val[["id","songs"]]
result_map=result[["id","songs"]]

In [6]:
check_h(val_map, result_map)

       id                                              songs
0  118598  [373313, 151080, 275346, 696876, 165237, 52593...
1  131447                                                 []
2   51464  [529437, 516103, 360067, 705713, 226062, 37089...
3   45144  [589668, 21711, 570151, 320043, 13930, 599327,...
4   79929  [672718, 121924, 102694, 683657, 201558, 38511...
       id                                              songs
0  118598  [207912, 623047, 422438, 703323, 638488, 32221...
1  131447  [16382, 278524, 196599, 442352, 704484, 589785...
2   51464  [456474, 533739, 680911, 596279, 425043, 45750...
3   45144  [144663, 367963, 174749, 377243, 7070, 64052, ...
4   79929  [211220, 412769, 445299, 106129, 70314, 242124...
val (23015, 2)
result (23015, 2)


In [7]:
val_map_unnest = np.dstack(
    (
        np.repeat(val_map.id.values, list(map(len, val_map.songs))),
        np.concatenate(val_map.songs.values)
    )
)
val_map_pre= pd.DataFrame(data = val_map_unnest[0], columns = val_map.columns)
del val_map_unnest
val_map_len = val_map_pre.groupby('id').songs.nunique().reset_index(name='len_val')
val_map_song= val_map_pre.groupby('songs').id.nunique().reset_index(name="song_val_n")
val_map_len=val_map_len.astype(int)
val_map_song=val_map_song.astype(int)

In [8]:
result_map_unnest = np.dstack(
    (
        np.repeat(result_map.id.values, list(map(len, result_map.songs))),
        np.concatenate(result_map.songs.values)
    )
)
result_map_len= pd.DataFrame(data = result_map_unnest[0], columns = result_map.columns)
del result_map_unnest
result_map_len = result_map_len.groupby('id').songs.nunique().reset_index(name='len_result')
result_map_len=result_map_len.astype(int)

In [9]:
check_h(val_map_len, result_map_len)

   id  len_val
0   3       66
1  10       22
2  18       23
3  23       14
4  28      100
   id  len_result
0   3         100
1  10         100
2  18         100
3  22         100
4  23         100
val (18636, 2)
result (23015, 2)


In [10]:
result_err=result_map_len[result_map_len["len_result"] <100]
result_err.shape

(88, 2)

In [11]:
err_merge = pd.merge(result_err, val_map_len)

In [12]:
err=pd.merge(err_merge, val_map)

In [13]:
err

,id,len_result,len_val,songs
0,490,7,5,"[381653, 566122, 158539, 261549, 368705]"
1,3991,6,5,"[405485, 381653, 144239, 261549, 343146]"
2,6114,93,42,"[626369, 653601, 105140, 418905, 38261, 199262..."
3,7857,66,38,"[38261, 199262, 396828, 487911, 448116, 341513..."
4,8610,70,37,"[570836, 209135, 105140, 199262, 422915, 54796..."
5,8912,7,5,"[381653, 566122, 261549, 368705, 343146]"
6,10658,92,37,"[266761, 621927, 464821, 264957, 669557, 15464..."
7,11670,6,5,"[405485, 566122, 158539, 686976, 368705]"
8,14962,19,5,"[405485, 381653, 546313, 368705, 343146]"
9,15142,6,5,"[405485, 566122, 686976, 144239, 261549]"


In [22]:
err.to_json("err.json")

In [14]:
err_map =  err[["id","songs"]]
err_map_unnest =  np.dstack(( np.repeat(err.id.values, list(map(len, err.songs))),
                             np.concatenate(err.songs.values)
                            ))
err_map_song =  pd.DataFrame(err_map_unnest[0], columns= err_map.columns)
del err_map_unnest


In [15]:
err_map_song =  err_map_song.groupby("songs").id.nunique().reset_index(name="song_err_n")

In [16]:
err_map_song

,songs,song_err_n
0,13281,1
1,15318,21
2,19436,1
3,19811,23
4,31344,17
5,32443,5
6,37748,1
7,38236,15
8,38261,19
9,39457,1


In [17]:
err_map_song[err_map_song["song_err_n"] > 20]

,songs,song_err_n
1,15318,21
3,19811,23
11,42155,21
18,70549,21
39,157055,22
41,166841,22
43,187047,28
46,209135,23
47,209993,24
50,222920,21


In [18]:
pd.merge(err_map_song, val_map_song) 

,songs,song_err_n,song_val_n
0,13281,1,127
1,15318,21,106
2,19436,1,3
3,19811,23,73
4,31344,17,56
5,32443,5,19
6,37748,1,98
7,38236,15,32
8,38261,19,94
9,39457,1,3


In [19]:
songmeta = pd.read_json("song_meta.json", encoding = "utf-8")

In [20]:
songmeta[songmeta["id"]==668128]

,album_id,album_name,artist_id_basket,artist_name_basket,id,issue_date,song_gn_dtl_gnr_basket,song_gn_gnr_basket,song_name
668128,10142592,사랑이 그리워,[247893],[태인],668128,20180227,"[GN0105, GN0101]",[GN0100],사랑이 그리워
